https://www.reddit.com/r/LangChain/comments/13jd9wo/improving_the_quality_of_qa_with_pdf/

In [22]:
from dotenv import load_dotenv

In [23]:
if not load_dotenv():
    print("Cannot load .env file. Environment file is not exists or not readable")
    exit(1)

In [24]:
from langchain.document_loaders import PyPDFLoader

In [25]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document


In [26]:
import os
import glob
from typing import List

In [27]:
# handle unstructured table

from langchain.document_loaders.parsers.pdf import PDFPlumberParser, PDFMinerParser

In [28]:
from langchain.document_loaders import (
    CSVLoader,
    EverNoteLoader,
    TextLoader,
    UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredODTLoader,
    UnstructuredPowerPointLoader,
    UnstructuredWordDocumentLoader,
    PyMuPDFLoader,
    PyPDFLoader
)

In [29]:
LOADER_MAPPING = {
    ".csv": (CSVLoader, {}),
    ".doc": (UnstructuredWordDocumentLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".enex": (EverNoteLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".md": (UnstructuredMarkdownLoader, {}),
    ".odt": (UnstructuredODTLoader, {}),
    ".pdf": (PyPDFLoader, {}),
    ".ppt": (UnstructuredPowerPointLoader, {}),
    ".pptx": (UnstructuredPowerPointLoader, {}),
    ".txt": (TextLoader, {"encoding": "utf8"})
}

In [30]:
def load_single_document(file_path: str) -> List[Document]:
    ext = "." + file_path.rsplit(".", 1)[-1].lower()
    if ext in LOADER_MAPPING:
        loader_class, loader_args = LOADER_MAPPING[ext]
        loader = loader_class(file_path, **loader_args)
        return loader.load()

    raise ValueError(f"Unsupported file extension '{ext}'")

In [31]:
from multiprocessing import Pool
from tqdm import tqdm

In [32]:
def load_documents(source_dir: str, ignored_files: List[str] = []) -> List[Document]: 
    all_files = []
    for ext in LOADER_MAPPING:
        all_files.extend(
            glob.glob(os.path.join(source_dir, f"**/*{ext.lower()}"), recursive=True)
            )
    filtered_files = [file_path for file_path in all_files if file_path not in ignored_files]

    results = []
    with tqdm(total=len(filtered_files), desc='Loading new documents', ncols=80) as pbar:
        for file_path in filtered_files:
            docs = load_single_document(file_path)
            if docs:
                results.extend(docs)
                pbar.update()

    return results

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

def get_text_splitter(chunk_size : int = 500, chunk_overlap : int =0, separators : list[str] = ["\n"], type : str = 'recursive'):
    
    if type == "char":
        text_splitter = CharacterTextSplitter(
            separators=separators,
            chunk_size=chunk_size, 
            chunk_overlap=chunk_overlap,
            length_function=len)
    else:
        text_splitter = RecursiveCharacterTextSplitter(
            separators=separators,
            chunk_size=chunk_size, 
            chunk_overlap=chunk_overlap,
            length_function=len)
    
    return text_splitter


In [34]:
# Load environment variables
persist_directory = os.environ.get('PERSIST_DIRECTORY')
source_directory = os.environ.get('SOURCE_DIRECTORY', 'src_documents')
embeddings_model_name = os.environ.get('EMBEDDINGS_MODEL_NAME')

In [35]:
def process_documents(chunk_size: int = 500, chunk_overlap: int = 20, ignored_files: List[str] = []) -> List[Document]:
    """
    Load documents and split in chunks
    """
    print(f"Loading documents from {source_directory}")
    documents = load_documents(source_directory, ignored_files)
    documents = list(filter(None, documents))
    if not documents:
        print("No new documents to load")
        return None
    print(f"Loaded {len(documents)} new documents from {source_directory}")
    print(documents)
    text_splitter = get_text_splitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    texts = text_splitter.split_documents(documents)
    print(f"Split into {len(texts)} chunks of text (max. {chunk_size} tokens each)")
    
    return texts

In [36]:
def vectorstore_exist(persist_directory: str, embeddings: HuggingFaceEmbeddings, settings) -> bool:
    """
    Checks if vectorstore exists
    """
    db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=settings)
    if not db.get()['documents']:
        return False
    return True

In [37]:
from constants import CHROMA_SETTINGS
import chromadb

In [38]:
def ingestion():
    # Create embeddings
    embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
    # Chroma persistent client
    chroma_client = chromadb.PersistentClient(settings=CHROMA_SETTINGS , path=persist_directory)

    if vectorstore_exist(persist_directory, embeddings, CHROMA_SETTINGS):
        # update vector db
        print(f"Appending to existing vectorstore at {persist_directory}")
        db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=CHROMA_SETTINGS, client=chroma_client)
        collection = db.get()
        print(collection)
        texts = process_documents(
            chunk_size=800,
            chunk_overlap=50,
            ignored_files=[metadata['source'] for metadata in collection['metadatas']]
            )
        if texts is not None:
            print("Creating embeddings...")
            db.add_documents(texts)
    else:
        # Create and store in vector db
        print("Creating new vectorstore")
        texts = process_documents(
            chunk_size=800,
            chunk_overlap=50
            )
        print("Creating embeddings...")
        db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory, client_settings=CHROMA_SETTINGS, client=chroma_client)

    if texts:
        db.persist()
        db = None

        print("Ingestion complete.")
    else:
        print("Nothing to ingest.")

In [39]:
ingestion()

Appending to existing vectorstore at db
{'ids': ['11e4b188-a00d-11ee-8b75-047c164e49b3', '11e4b189-a00d-11ee-bed0-047c164e49b3', '11e4b18a-a00d-11ee-87b5-047c164e49b3', '11e4b18b-a00d-11ee-b42f-047c164e49b3', '11e4b18c-a00d-11ee-be11-047c164e49b3', '11e4b18d-a00d-11ee-9b1d-047c164e49b3', '11e4b18e-a00d-11ee-b541-047c164e49b3', '11e4b18f-a00d-11ee-8af4-047c164e49b3', '11e4b190-a00d-11ee-ace5-047c164e49b3', '11e4b191-a00d-11ee-bfce-047c164e49b3', '11e4b192-a00d-11ee-9894-047c164e49b3', '11e4b193-a00d-11ee-8e81-047c164e49b3', '11e4b194-a00d-11ee-af69-047c164e49b3', '11e4b195-a00d-11ee-810d-047c164e49b3', '11e4b196-a00d-11ee-bc85-047c164e49b3', '11e4b197-a00d-11ee-9615-047c164e49b3', '11e4b198-a00d-11ee-aa4f-047c164e49b3', '11e4b199-a00d-11ee-af7f-047c164e49b3', '11e4b19a-a00d-11ee-8f74-047c164e49b3', '11e4b19b-a00d-11ee-8daa-047c164e49b3', '11e4b19c-a00d-11ee-9f0e-047c164e49b3', '11e4b19d-a00d-11ee-a05a-047c164e49b3', '11e4b19e-a00d-11ee-b135-047c164e49b3', '11e4b19f-a00d-11ee-b912-047c16

Loading new documents: 0it [00:00, ?it/s]

No new documents to load
nothing to ingest
